In [1]:
import sqlite3
import hashlib
import tkinter as tk
from tkinter import messagebox, ttk
from PIL import Image, ImageTk
import os

# --- Database setup ---
def create_tables():
    conn = sqlite3.connect('inventory_master.db')  # Updated DB name
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    username TEXT NOT NULL,
                    password TEXT NOT NULL)''')
    c.execute('''CREATE TABLE IF NOT EXISTS products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    quantity INTEGER NOT NULL,
                    price REAL NOT NULL)''')
    conn.commit()
    conn.close()

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def add_user(username, password):
    conn = sqlite3.connect('inventory_master.db')
    c = conn.cursor()
    hashed_password = hash_password(password)
    c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
    conn.commit()
    conn.close()

def authenticate_user(username, password):
    conn = sqlite3.connect('inventory_master.db')
    c = conn.cursor()
    hashed_password = hash_password(password)
    c.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, hashed_password))
    user = c.fetchone()
    conn.close()
    return user is not None

def get_products():
    conn = sqlite3.connect('inventory_master.db')
    c = conn.cursor()
    c.execute("SELECT * FROM products")
    items = c.fetchall()
    conn.close()
    return items

def add_product(name, quantity, price):
    conn = sqlite3.connect('inventory_master.db')
    c = conn.cursor()
    c.execute("INSERT INTO products (name, quantity, price) VALUES (?, ?, ?)", (name, quantity, price))
    conn.commit()
    conn.close()

def update_product(product_id, name, quantity, price):
    conn = sqlite3.connect('inventory_master.db')
    c = conn.cursor()
    c.execute("UPDATE products SET name=?, quantity=?, price=? WHERE id=?", (name, quantity, price, product_id))
    conn.commit()
    conn.close()

def delete_product(product_id):
    conn = sqlite3.connect('inventory_master.db')
    c = conn.cursor()
    c.execute("DELETE FROM products WHERE id=?", (product_id,))
    conn.commit()
    conn.close()

def add_default_user():
    if not authenticate_user('hermoine', 'admin123'):
        add_user('hermoine', 'admin123')

# --- GUI ---
class InventoryApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Inventory Management System")
        self.root.geometry("700x550")
        self.root.config(bg="#2c3e50")

        self.show_password = False
        self.create_login_screen()

    def toggle_password(self):
        if self.show_password:
            self.password_entry.config(show="*")
            self.eye_button.config(image=self.eye_icon)
        else:
            self.password_entry.config(show="")
            self.eye_button.config(image=self.eye_slash_icon)
        self.show_password = not self.show_password

    def create_login_screen(self):
        self.login_frame = tk.Frame(self.root, bg="#34495e", padx=30, pady=30)
        self.login_frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        tk.Label(self.login_frame, text="Username", bg="#34495e", fg="white", font=("Arial", 12)).grid(row=0, column=0, sticky='e', pady=8, padx=5)
        self.username_entry = tk.Entry(self.login_frame, font=("Arial", 12))
        self.username_entry.grid(row=0, column=1, pady=8, columnspan=2)

        tk.Label(self.login_frame, text="Password", bg="#34495e", fg="white", font=("Arial", 12)).grid(row=1, column=0, sticky='e', pady=8, padx=5)
        self.password_entry = tk.Entry(self.login_frame, show="*", font=("Arial", 12))
        self.password_entry.grid(row=1, column=1, pady=8, sticky='w')

        try:
            self.eye_icon = ImageTk.PhotoImage(Image.open("eye.png").resize((20, 20)))
            self.eye_slash_icon = ImageTk.PhotoImage(Image.open("eye_slash.png").resize((20, 20)))
        except FileNotFoundError:
            self.eye_icon = self.eye_slash_icon = None

        self.eye_button = tk.Button(self.login_frame, image=self.eye_icon, command=self.toggle_password, bg="#34495e", borderwidth=0)
        self.eye_button.grid(row=1, column=2, padx=5)

        tk.Button(self.login_frame, text="Login", width=25, command=self.authenticate,
                  bg="#1abc9c", fg="white", font=("Arial", 12, "bold"))\
                  .grid(row=2, column=0, columnspan=3, pady=15)

    def authenticate(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        if authenticate_user(username, password):
            self.create_inventory_screen()
        else:
            messagebox.showerror("Login Failed", "Invalid username or password.")

    def create_inventory_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

        tk.Label(self.root, text="Inventory Dashboard", bg="#2c3e50", fg="white", font=("Helvetica", 16, "bold")).pack(pady=10)

        self.tree = ttk.Treeview(self.root, columns=("ID", "Name", "Quantity", "Price"), show="headings")
        for col in ("ID", "Name", "Quantity", "Price"):
            self.tree.heading(col, text=col)
        self.tree.pack(fill="both", expand=True, padx=10, pady=10)

        form = tk.Frame(self.root, bg="#2c3e50")
        form.pack(pady=10)

        tk.Label(form, text="Name", bg="#2c3e50", fg="white").grid(row=0, column=0, padx=5)
        self.name_entry = tk.Entry(form)
        self.name_entry.grid(row=0, column=1, padx=5)

        tk.Label(form, text="Qty", bg="#2c3e50", fg="white").grid(row=1, column=0, padx=5)
        self.qty_entry = tk.Entry(form)
        self.qty_entry.grid(row=1, column=1, padx=5)

        tk.Label(form, text="Price", bg="#2c3e50", fg="white").grid(row=2, column=0, padx=5)
        self.price_entry = tk.Entry(form)
        self.price_entry.grid(row=2, column=1, padx=5)

        btn_frame = tk.Frame(self.root, bg="#2c3e50")
        btn_frame.pack()

        tk.Button(btn_frame, text="Add", command=self.add_product, width=10).grid(row=0, column=0, padx=5)
        tk.Button(btn_frame, text="Update", command=self.update_product, width=10).grid(row=0, column=1, padx=5)
        tk.Button(btn_frame, text="Delete", command=self.delete_product, width=10).grid(row=0, column=2, padx=5)
        tk.Button(btn_frame, text="Stock Summary", command=self.show_stock_summary, bg="orange", width=15).grid(row=0, column=3, padx=5)
        tk.Button(btn_frame, text="Low Stock Alert", command=self.show_low_stock_alert, bg="red", fg="white", width=15).grid(row=0, column=4, padx=5)

        self.tree.bind("<ButtonRelease-1>", self.fill_form_from_selection)

        self.refresh_products()

    def refresh_products(self):
        for row in self.tree.get_children():
            self.tree.delete(row)
        for product in get_products():
            self.tree.insert("", "end", values=product)

    def fill_form_from_selection(self, event):
        selected = self.tree.focus()
        if selected:
            values = self.tree.item(selected, 'values')
            self.name_entry.delete(0, tk.END)
            self.name_entry.insert(0, values[1])
            self.qty_entry.delete(0, tk.END)
            self.qty_entry.insert(0, values[2])
            self.price_entry.delete(0, tk.END)
            self.price_entry.insert(0, values[3])

    def add_product(self):
        name = self.name_entry.get()
        qty = self.qty_entry.get()
        price = self.price_entry.get()
        if name and qty.isdigit() and price.replace('.', '', 1).isdigit():
            add_product(name, int(qty), float(price))
            self.refresh_products()
        else:
            messagebox.showwarning("Invalid Input", "Enter valid product details.")

    def update_product(self):
        selected = self.tree.focus()
        if selected:
            values = self.tree.item(selected, 'values')
            product_id = values[0]
            name = self.name_entry.get()
            qty = self.qty_entry.get()
            price = self.price_entry.get()
            if name and qty.isdigit() and price.replace('.', '', 1).isdigit():
                update_product(product_id, name, int(qty), float(price))
                self.refresh_products()
            else:
                messagebox.showwarning("Invalid Input", "Enter valid product details.")

    def delete_product(self):
        selected = self.tree.focus()
        if selected:
            values = self.tree.item(selected, 'values')
            product_id = values[0]
            delete_product(product_id)
            self.refresh_products()

    def show_stock_summary(self):
        products = get_products()
        total_products = len(products)
        total_items = sum(p[2] for p in products)
        total_value = sum(p[2] * p[3] for p in products)
        low_stock = [f"{p[1]} (Qty: {p[2]})" for p in products if p[2] < 5]

        summary = f"Total Products: {total_products}\nTotal Items in Stock: {total_items}\nTotal Inventory Value: ₹{total_value:.2f}\n"
        if low_stock:
            summary += "\nLow Stock:\n" + "\n".join(low_stock)

        messagebox.showinfo("Stock Summary", summary)

    def show_low_stock_alert(self):
        products = get_products()
        low_stock = [f"{p[1]} (Qty: {p[2]})" for p in products if p[2] < 5]
        if low_stock:
            messagebox.showwarning("Low Stock Alert", "\n".join(low_stock))
        else:
            messagebox.showinfo("Stock Alert", "All products are sufficiently stocked.")

# --- Main ---
if __name__ == "__main__":
    create_tables()
    add_default_user()
    root = tk.Tk()
    app = InventoryApp(root)
    root.mainloop()
